In [ ]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython.display

In [ ]:
def plot_spectrogram(y, sr):
    hop_length = int(4096 * 0.4)
    plt.figure(figsize=(15, 8))
    signal = librosa.stft(y, n_fft=4096, hop_length=hop_length, window='hamming')
    signal = librosa.amplitude_to_db(np.abs(signal) ** 2)
    librosa.display.specshow(signal, sr=sr, hop_length=hop_length , x_axis="time", y_axis='log', cmap='magma')
    # plt.colorbar(format="%+2.f")

def running_mean(x, n):
    if n % 2 == 0:
        raise ValueError("Filter length is not odd")

    aver = np.convolve(x, np.ones((n,)) / n, mode='same')
    n_half = int((n - 1) / 2)
    corr_length = np.array(range(n_half + 1, n)) / n
    aver[:n_half] = np.divide(aver[:n_half], corr_length)
    aver[-n_half:] = np.divide(aver[-n_half:], corr_length[::-1])

    return aver

def filter_and_normalize(x, filts_len):
    for filt_len in filts_len:
        x = running_mean(x, filt_len)
    
    x -= np.min(x)
    x = x / np.max(x)

    return x

def plot_signal(y, sr, f_min=0, f_max=int(1e8)):
    signal = librosa.stft(y, n_fft=4096, hop_length=int(4096 * 0.4), window='hamming')
    signal = np.abs(signal) ** 2
    
    signal = signal[f_min:f_max].sum(0)
    signal = filter_and_normalize(signal, (11, 5))
    
    x = np.linspace(0, len(y), len(signal))
    y3 = np.interp(range(len(y)), x, signal)
    
    fig, ax1 = plt.subplots(figsize=(15, 8))
    ax2 = ax1.twinx()
    ax1.plot(range(len(y3)), y, c='g', alpha=0.2)
    ax2.plot(range(len(y3)), y3, c='black')
    
    ax1.set_ylabel('Amplitude')
    ax1.set_xlabel('Time')


In [ ]:
audio_path = 'output.mp3'

y, sr = librosa.load(audio_path, sr=None)

In [ ]:
FROM_MINUTE = 0
FROM_SECOND = 0

TO_MINUTE = 1
TO_SECOND = 0

convert = lambda m, s: m * 60 + s

FROM_TIME = convert(FROM_MINUTE, FROM_SECOND)
TILL_TIME = convert(TO_MINUTE, TO_SECOND)

FROM_SAMPLE = FROM_TIME * sr
TILL_SAMPLE = TILL_TIME * sr

y1 = y[FROM_SAMPLE: TILL_SAMPLE]

In [ ]:
plot_spectrogram(y1, sr)

In [ ]:
plot_signal(y1, sr, 550)

In [ ]:
plot_signal(y1, sr, 0)